In [66]:
# Imports of all packages used trough out this tutorial
%matplotlib inline
import os
import copy
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import rv_discrete

# Recreating the AW-SGD Paper

In this notebook I try to replicate the results of the paper "Accerlerating Stochastic Gradient Descent by Online learning to Sample".

## Least Squares Matrix Factorisation

The first experiment in the paper that I will attempt to replicate is low rank matrix factorisation by minimisation of the sum square loss between the elements of the reconstructed matrix and the original matrix.


In [84]:
# Define Some Small and Useful Functions
def matrix_square_loss(U,V,Y):
    return np.sum((np.dot(U,V.T) - Y)**2)
    
def grad_u(U,V,Y,i,j):
    """ Gradient of loss with respect to an element of U
    """
    return 2*(np.dot(U[i,:],V[j,:].T) - Y[i,j])*V[j,:]

def grad_v(U,V,Y,i,j):
    """ Gradient of loss with respect to an element of V
    """
    return 2*(np.dot(U[i,:],V[j,:].T) - Y[i,j])*U[i,:]

def one_hot(i,N):
    """ Return a vector of length N thats all 0's apart from at place i
    """
    x = np.zeros(N)
    x[i] = 1
    return x

def softmax(tau):
    return np.exp(tau)/np.sum(np.exp(tau))


# Some functions for sampling from discrete distributions
class distribution(rv_discrete):
    
    def plot(self,N):
        xk = np.arange(N)
        fig, ax = plt.subplots(1, 1)
        ax.plot(xk, self.pmf(xk), 'ro', ms=12, mec='r')
        ax.vlines(xk, 0, self.pmf(xk), colors='r', lw=4)
        
def create_dist(tau):
    xk = np.arange(np.size(tau))
    pk = softmax(tau)
    custm = distribution(name='custm', values=(xk, pk))
    return custm

# Define Updates for SGD and AWSGD
def sgd_update(U,V,Y,i,j,mu):
    """ One Update of SGD
    """
    u_new = U[i,:] - mu*grad_u(U,V,Y,i,j)
    v_new = V[j,:] - mu*grad_v(U,V,Y,i,j)
    return u_new,v_new

def AW_sgd_update(U,V,Y,i,j,mu,dist_u,dist_v,rho,tau1,tau2):
    """ One update of Awsgd
    """
    du = grad_u(U,V,Y,i,j)/dist_u._pmf(i)
    dv = grad_v(U,V,Y,i,j)/dist_v._pmf(j)
    u_new = U[i,:] - mu*du
    v_new = V[j,:] - mu*dv
    dtau1 = one_hot(i,N) - softmax(tau1)
    dtau2 = one_hot(j,M) - softmax(tau2)
    tau1 = tau1 + rho*((np.linalg.norm(du))**2)*dtau1
    tau2 = tau2 + rho*((np.linalg.norm(dv))**2)*dtau2
    return u_new,v_new,tau1,tau2

    

In [125]:
# Define functions to perform SGD and AWSGD

def factor_sgd(M,k,num_iter,initial_mu):
    """ Takes in a Matrix M and returns two matrices A,B such that M ~= AB'
        A and B are found by minimising the square loss
        
        Inputs
        ------
            M : type - np.array
                Matrix to be factorised
            k:  type - int
                Rank of approximated matrix
      num_iter: type - int
     Inital_mu: type - float
                Initial learning rate
        
       OutPuts
       -------
           A,B: np.array
                Factorised matrices
        errors: np.array
                Square error as a function of iteration number
    """
    
    # Initialse the factored matrices randomly    
    num_rows = np.shape(M)[0]
    num_cols = np.shape(M)[1]
    A = np.random.randn(num_rows,k)
    B = np.random.randn(num_cols,k)

    # Storage for the error
    errors = np.zeros(num_iter)
                
    for step in range(num_iter):
        # Define Learning Rate
        rate = initial_mu/(num_rows/2.0 + 1.0*step)
        # Sample a row and a column
        i,j = np.random.randint(num_rows), np.random.randint(num_cols)
        # Perform one step of sgd
        A[i,:],B[j,:] = sgd_update(A,B,M,i,j,rate)
        # Calculate and store the error
        errors[step] = matrix_square_loss(A,B,M)
    
    return A,B,errors
        

In [126]:
def factor_aswsgd(M,k,num_iter,initial_mu,initial_tau = None):
    """ Takes in a Matrix M and returns two matrices A,B such that M ~= AB'
        A and B are found by minimising the square loss
        
        Inputs
        ------
            M : type - np.array
                Matrix to be factorised
            k:  type - int
                Rank of approximated matrix
      num_iter: type - int
     Inital_mu: type - float
                Initial learning rate
        
       OutPuts
       -------
           A,B: np.array
                Factorised matrices
        errors: np.array
                Square error as a function of iteration number
    """
    
    # Initialse the factored matrices randomly    
    num_rows = np.shape(M)[0]
    num_cols = np.shape(M)[1]
    A = np.random.randn(num_rows,k)
    B = np.random.randn(num_cols,k)
    
    # Storage for the error
    errors = np.zeros(num_iter)
    
    #Initialise Tau if not already initialised
    if initial_tau is None:
        tau_rows = np.random.randn(num_rows)
        tau_cols = np.random.randn(num_cols)
    else:
        tau_rows = initial_tau[0:num_rows]
        tau_cols = initial_tau[num_rows + 1:]
    
    for step in range(num_iter):
        # Define the leanring rates
        rate = initial_mu/(num_rows/2.0 + 1.0*step)
        rho = initial_mu/100000.0
        # Sample a row and a column from a soft-max parameterised by tau
        row_dist = create_dist(tau_rows)
        col_dist = create_dist(tau_cols)
        i,j = row_dist.rvs(),col_dist.rvs()
        # Perform one step of awsgd
        A[i,:],B[j,:],tau_rows,tau_cols = AW_sgd_update(A,B,M,i,j,rate,row_dist,col_dist,rho,tau_rows,tau_cols)
        # Calculate and return the Error
        errors[step] = matrix_square_loss(A,B,M)
    
    return A,B,errors,row_dist,col_dist

## Experiments with matrix factorisation SGD vs AW-SGD

In the original paper, the authors provide two demonstrations of awsgd for matrix factorisation. First on a randomly generated low rank matrix of dimension 100x100 with rank 10 and second on mnist.

For the first experiment the model learning rate was set to $\frac{\rho_0}{0.5N +t}$ and the sampling learning rate was kept constant. The results of each experiment were averaged over 10 runs.

In [128]:
# Generate a Low Rank Matrix Y = UV'
N = 100 # Dimension 1 of M
M = 100 # Dimension 2 0f M 
K = 10 # Rank of the matrix

U = np.random.randn(N,K)
V = np.random.randn(M,K)
Y = np.dot(U,V.T)


In [162]:
# Perform 10 rounds of sgd and awsgd

num_iter = 1000
average_sgd_error = np.zeros(num_iter)
average_awsgd_error = np.zeros(num_iter)

initial_mu_sgd = 5.0
initial_mu_awsgd = 0.001

for iter in range(10):
    # sgd
    #_,_,esgd = factor_sgd(Y,10,num_iter,initial_mu_sgd)
    #average_sgd_error = average_sgd_error + esgd
    #awsgd
    _,_,eawsgd,_,_ = factor_aswsgd(Y,10,num_iter,initial_mu_awsgd,)
    


plt.plot(eawsgd/10,c = 'blue')




/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:37: RuntimeWarning: divide by zero encountered in divide
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:43: RuntimeWarning: invalid value encountered in multiply


KeyError: nan

In [ ]:
# Plot and compare the average error